# Optimization of linear signal processing in photon counting lidar
This notebook was developed as a suppliment to the publication

Matthew Hayman, Robert A. Stillwell and Scott M. Spuler, 
"Optimization of linear photon counting lidar signal processing through Poisson thinning,"
submitted to Optics Letters *In Review*, DOI:TBD

The notebook demonstrates the basic concepts for optimizing linear smoothing
kernenls to a particular lidar scene by splitting an observed photon count
profile into a "fit" and "verification" profile.  Smoothing kernels are applied
to the "fit" profile then evaluated against the verification data set to find 
the optimal filter kernel.

The code and functions contained in this notebook are available for public use
so long as the original publication is referenced, but in the interest of independent verification, we strongly recommend that you write your own functions.  While every effort is made to provide accurate calcualtions, this code is not guaranteed to be free from errors.

The data used in this example is from an NCAR MicroPulse DIAL (MPD) [1,2] using the potassium HSRL channels [3].  This 
data is provided for example purposes only and should not be used in scientific
study.  No data quality assurance can be provided for these datasets.

1. Spuler et al., "Field-deployable diode-laser-based differential absorption lidar (DIAL) for profiling water vapor," Atmos. Meas. Tech., 8, 1073–1087, DOI:10.5194/amt-8-1073-2015 (2015).

2. NCAR/EOL Remote Sensing Facility, "NCAR MPD data. Version 1.0,"Retrieved 9 Jan 2020, DOI:10.26023/MX0D-Z722-M406.

3. Stillewell et al.,"Demonstration of a combined differential absorption and high spectral resolution lidar for profiling atmospheric temperature," Opt. Express, 28, 71-93, DOI: 10.1364/OE.379804 (2020).

In [ ]:
# import python libraries
import os
import sys
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
import xarray as xr  # not part of the standard python environemnt. if using anaconda: conda install xarray

from matplotlib.colors import LogNorm
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

In [ ]:
data_path = '../data/'

In [ ]:
ncfile = 'mpd05.20181022T12300019921_20181022T15163019921.nc'
# ncfile = 'mpd05.20181022T18032019921_20181022T20495019531.nc'

In [ ]:
# list of lidar profile names to be loaded
lidar_profile_data = ['Combined_Counts','Molecular_Counts']

In [ ]:
"""
Function definitions for optimizing the filter
"""

def poisson_thin(pc_data):
    """
    Poisson thin photon counting data
    
    inputs:
        pc_data - array of raw photon count data
            This is assumed to have a Poisson PDF
    outputs:
        pc_1,pc_2 - the resulting thinned photon count
            arrays
    """
    # the first thinned profile is calculated using a binomial random
    # number generator to simulate 
    # flipping a coin to decide if a photon count is included in this
    # profile or not
    pc_1 = np.random.binomial(pc_data.astype(np.int),0.5,size=pc_data.shape)
    # the second profile is whatever photon counts are left
    pc_2 = pc_data-pc_1
    
    return pc_1,pc_2

def build_Gaussian_kernel(sigt,sigz,norm=True):
    """
    Generates a Gaussian convolution kernel for
    standard deviations sigt and sigz in units of grid points.
    sigt and sigz are defined in units of grid steps
    """        

    
    nt = np.round(4*sigt) # estimate size of time grid needed
    nz = np.round(4*sigz) # estimate size of range grid needed
    t = np.arange(-nt,nt+1) # create time grid     
    z = np.arange(-nz,nz+1) # create range grid
    
    
    # build Gaussian kernel in time
    kconv_t = np.exp(-t**2*1.0/(sigt**2))
    
    # check for singularities in the definition
    # if they exist, make the filter a delta function in time
    if kconv_t.size > 1:
        if np.sum(kconv_t) == 0:
            it0 = np.argmin(np.abs(t))
            kconv_t[it0] = 1.0
    else: 
        kconv_t = np.ones(1)

    # build Gaussian kernel in range
    kconv_z = np.exp(-z**2*1.0/(sigz**2))
    
    # check for singularities in the definition
    # if they exist, make the filter a delta function in range
    if kconv_z.size > 1:
        if np.sum(kconv_z) == 0:
            iz0 = np.argmin(np.abs(z))
            kconv_z[iz0] = 1.0
    else:
        kconv_z = np.ones(1)

    # combine the time and range kernels
    kconv = kconv_t[:,np.newaxis]*kconv_z[np.newaxis,:]

    # normalize the area of the kernel to conserve energy
    if norm:
        kconv = kconv/(1.0*np.sum(kconv))

    return kconv

In [ ]:
"""
Load the photon counts from the netcdf file
"""


profs = {}
with xr.open_dataset(data_path+ncfile) as ds:
    for lvar in lidar_profile_data:
        profs[lvar] = ds[lvar].values
    lidar_range = ds['range'].values.copy()
    time = ds['time'].values.copy()
    

In [ ]:
"""
This notebook will run a demonstration case on the combined
channel of the MPD HSRL operating at 770.  This is effectively
equivilant to a backscatter lidar observation.
"""
demo_var = 'Combined_Counts'

There are a variety of features in the scene from this data file.  Thick low clouds are between time indices 1000 and 2000.  High ice clouds are seen intermittantly starting around time index 2500 until lower clouds arrive late in the day near time index 7500.  The sun rises around time index 4800 and several instances of high background occur due to the simultaneous presence of sun and clouds.

In [ ]:
# plot a quicklook of the backscatter data
fig,ax = plt.subplots(1,1,figsize=(18,4))
ax.imshow(profs[demo_var][:,::-1].T,norm=LogNorm())

The demonstration presented here focuses on optimization of range smoothing kernels, but kernels can similarly be optimized for time smoothing or 2D kernels consisting of range and time components.  

Generally we perform both time and range optimization independently where each time bin has its own unique smoothing kernel and each range bin has its own uniquely determined time smoothing kernel.

In order to compare methodology, thinning is performed once and different methods are subsequently applied to that same data.  Comparing results between two different thinned cases, even if they originate from the same profile is generally not effective because the offset in the negative log-likelihood is prone to variation between different thinning operations.

In [ ]:
itime = 150   # select a profile index for this demonstration case 
              # interesting cases: 
              #   thin high cloud: 400
              #   thick cirrus: 150


max_alt = 12e3

imax_alt = np.argmin(np.abs(lidar_range-max_alt))
            
# assign the demonstration case to its own independent variable
pdemo = (profs[demo_var][itime,3:imax_alt])[np.newaxis,:]  # remove bottom three bins from laser "bang"
plidar_range = lidar_range[3:imax_alt]  # create a new range array for the bang-removed profile
dz = np.mean(np.diff(plidar_range))  # store the range resolution

# Poisson Thinning
Poisson thinning generates two statistically independent observations from one observation.  The resulting thinned profiles have the same underlying mean photon arrival rate driving the observations.  This is demonstrated below on the two thinned profiles are
```
pfit
```
used for applying the filter ($\mathbf{f}$ in the publication) and 
```
pver
```
used for scoring the the filtered result ($\mathbf{g}$ in the publication).

If the two observations have equal signals but uncorrelated noise, then
$$E\left[\frac{\mathbf{f}-\mathbf{g}}{\sqrt{\sigma_f^2+\sigma_g^2}}\right] \approx 0 $$
and 
$$std\left[\frac{\mathbf{f}-\mathbf{g}}{\sqrt{\sigma_f^2+\sigma_g^2}}\right] \approx 1 $$

where for a Poisson observation $f$, the variance in the estimate of mean photons is estimated by $\sigma_f^2 = f+1$

In [ ]:
# Poisson thin the raw photon counts of the profile
pfit,pver = poisson_thin(pdemo)

In [ ]:
# plot the thinned data
# the two verify and fit data should have 
# statistically independent noise but common 
# underlying signals
fig,ax = plt.subplots(1,1,figsize=(5,5))
ax.plot(pdemo.flatten(),plidar_range*1e-3,'k.',label='observation')
ax.plot(pfit.flatten(),plidar_range*1e-3,'.',alpha=0.5,label='fit data')
ax.plot(pver.flatten(),plidar_range*1e-3,'.',alpha=0.5,label='verify data')
ax.set_xscale('log')
ax.set_ylabel('Range [km]')
ax.set_xlabel('Photon Counts')
ax.grid(b=True)
ax.set_ylim([0,12])
ax.legend(loc=1)

The original photon count data, thinned fit data, and thinned verification data are shown above.  The shapes of the thinned profiles are the same but the statistical noise is not.

In [ ]:
# Plot the difference in photon counts between the
# fit and verify profiles 
# compare this to the expected standard deviation
# for uncorrelated Poisson observations
fig,ax = plt.subplots(1,1,figsize=(5,5))
ax.plot(pfit.flatten()-pver.flatten(),plidar_range,'g.',label='Actual')
ax.plot(np.sqrt(pfit+1+pver+1).flatten(),plidar_range,'k--',label='Expected std.')
ax.plot(-np.sqrt(pfit+1+pver+1).flatten(),plidar_range,'k--')
ax.set_ylabel('Range [m]')
ax.set_xlabel('Photon Count Difference')
ax.grid(b=True)
ax.set_ylim([0,12e3])
ax.legend(loc=2)

# adjusting by the expected uncertainty results in a
# a mean value near zero with
# a standard deviation near one
# further suggesting the signals are common but the
# noise is statistically independent
print('mean: %f'%np.mean((pfit-pver)/np.sqrt(pfit+1+pver+1)))
print('std: %f'%np.std((pfit-pver)/np.sqrt(pfit+1+pver+1)))

The above plot shows the difference between the two thinned profiles (green dots) with the theoretically estimated variance in the difference assuming both profiles are observations of Poisson random variables and they are uncorrelated. ($\sqrt{\sigma_f^2+\sigma_g^2}$).  The theoretically estimated standard deviation appears representative of the scatter in the difference signal and the scattered signal appears to be zero mean.

The standard deviation normalized mean and standard deviation are reported above these figures.  The mean is near zero and the standard deviation is near one.

# Find the Optimal Filter
The optimal filter kernel in a set is found by applying all the kernels to $\mathbf{f}$ then scoring the result $\tilde{\mathbf{f}}$ against the verification profile $\mathbf{g}$.  The score is calculated using the negative log-likelihood of a Poisson random variable
$$\mathcal{E} = \tilde{\mathbf{f}} - \mathbf{g}\ln \tilde{\mathbf{f}} $$
The filter that produces the lowest negative log-likelihood is taken to be the best filter from the set for this particular scene.

In [ ]:
"""
Filter the raw profile
"""

# define range of filters to try
filt_size = np.logspace(-1,1,40)

# initialize the output arrays
inv_ll = np.zeros(filt_size.size)
inv_ll_fit = np.zeros(filt_size.size)
filt_profs = np.zeros((filt_size.size,pfit.size))

for ifilt,filter_width in enumerate(filt_size):
    # get the convolution kernel
    kern = build_Gaussian_kernel(0,filter_width)
    
    # normalize by the amount of points in a region
    # to avoid edge effects
    norm = np.ones(pfit.shape)
    norm = scipy.signal.convolve2d(norm,kern,mode='same')
    # apply the filter and normalize the result
    # with the amount of data points contributing
    pfilt = scipy.signal.convolve2d(pfit,kern,mode='same')/norm
    
    pfilt[pfilt==0] = 0.001 # avoid zero values that blow up the log function
    
    # save the results
    filt_profs[ifilt,:] = pfilt
    inv_ll[ifilt] = np.nansum(pfilt-pver*np.log(pfilt))
    inv_ll_fit[ifilt] = np.nansum(pfilt-pfit*np.log(pfilt)) # for comparison, also store iLL using fit data as a basis

# get the index to the solution
isol = np.argmin(inv_ll)

In [ ]:
# plot the resulting optimized profile over the raw data
# also include an over-filtered example
# in a separate plot show the negative log-likelihood as a function of filter width

fig,ax = plt.subplots(2,1,figsize=(5,10))
ax[0].plot(pver.flatten(),plidar_range*1e-3,'.',markersize=2,label='raw data')
ax[0].plot(filt_profs[isol,:],plidar_range*1e-3,'k--',label='filtered')
ax[0].plot(filt_profs[-1,:],plidar_range*1e-3,':',label='over filtered')
ax[0].set_xscale('log')
ax[0].set_ylabel('Range [km]')
ax[0].set_xlabel('Photon Counts')
ax[0].grid(b=True)
ax[0].set_ylim([0,12])
ax[0].legend()

ax[1].plot(filt_size*dz,inv_ll*1e-3,'b.')
ax[1].plot(filt_size[isol]*dz,inv_ll[isol]*1e-3,'gs')
ax[1].plot(filt_size*dz,(inv_ll_fit-np.mean(inv_ll_fit)+np.mean(inv_ll))*1e-3,'rd')
ax[1].set_xscale('log')
ax[1].grid(b=True)
ax[1].set_xlabel('Filter Width [m]')
ax[1].set_ylabel('Negative Log-Likelihood x $10^{-3}$');


In [ ]:
"""
Plots for publication
"""

fig,ax = plt.subplots(1,1,figsize=(3.2,3.2))
ax.plot(pdemo.flatten(),plidar_range*1e-3,'k.',markersize=2,label='observation')
ax.plot(pfit.flatten(),plidar_range*1e-3,'.',markersize=2,alpha=0.5,label='fit data')
ax.plot(pver.flatten(),plidar_range*1e-3,'.',markersize=2,alpha=0.5,label='verify data')
ax.plot(filt_profs[isol,:],plidar_range*1e-3,'-',linewidth=1,label='filtered')
ax.set_xscale('log')
ax.set_ylabel('Range [km]')
ax.set_xlabel('Photon Counts')
ax.grid(b=True)
ax.set_ylim([0,12])
ax.legend(loc=3,prop={'size': 8})
# plt.savefig('../../plots/Thinned_and_Filt_Data.png',dpi=300,bbox_inches='tight')



In [ ]:
fig,ax = plt.subplots(1,1,figsize=(3.1,2.1))
ax.plot(filt_size*dz,inv_ll*1e-3,'b.',markersize=3,label='verification')
ylimits = ax.get_ylim()  # let the actual ILL data set the axes limits
ax.plot(filt_size[isol]*dz,inv_ll[isol]*1e-3,'gd',alpha=0.5)
ax.plot(filt_size*dz,(inv_ll_fit-np.mean(inv_ll_fit)+np.mean(inv_ll))*1e-3,'rs',markersize=2,label='fit')
ax.set_ylim(ylimits)
ax.set_xscale('log')
ax.grid(b=True)
ax.set_xlabel('Filter Width [m]',fontsize=8)
ax.set_ylabel('Negative Log-Likelihood',fontsize=8)
ax.legend()
# plt.savefig('../../plots/InvLL_Filter.png',dpi=300,bbox_inches='tight')

In [ ]:
filt_size[isol]*dz

The top figure shows the verification data in blue dots and the optimally filtered data is in the blacked dashed line.  For comparison, an over filtered case is also shown as the orange dotted line.  Overfiltering is more effective as supressing the random errors, but also skews the signal, particularly where clouds are present.

The bottom figure shows the negative log-likelihood for each filter kernel width (defined by the standard deviation of a Gaussian) where the minimum (optimal) value is indicated by the green diamond.

# Synthetic Data

To see the value of Poisson thinning and verification with the Negative Log-likelihood, we use synthetic data.  Here we know the true signal and can compare the results obtained using Poisson thinning.

We generate a relatively simple 1D profile where aerosols exist below 3 km and there is some enhancement in aerosol near the top of the boundary layer due to entrainment (similar to the true profile we saw earlier).

In [ ]:
dz=37.5
lidar_range_s = np.arange(dz*2,12037.5,dz)[np.newaxis,:]
bg_s = 1e-8

mult = 6e8

smol = np.exp(-lidar_range_s/8e3)
saer = np.zeros(lidar_range_s.shape)
saer[lidar_range_s < 3e3] = 4
saer += 10*np.exp(-(lidar_range_s-3e3)**2/80**2)

# add cloud
# Randomly generated cloud structure
nstruc = (np.maximum(1,np.random.randn()*10+5)).astype(np.int) # generate number of structures
calt = np.random.randn(nstruc)*1e3+7e3  # generate altitude of the structures centered on 7km
csig = np.maximum(50,np.random.randn(nstruc)*0.2e3+0.2e3)  # generate structure widths
cBS = np.maximum(0,np.random.randn(nstruc)*30+30)  # generate the structure backscatter

# cloud data used for publication
# generated by the above random structure generator
# comment this block to use a randomly generated structure
nstruc = 4
calt = np.array([5192.16337811, 7690.21222657, 7882.29934153, 7531.27967589])
csig = np.array([ 50., 232.4572076,  331.93039876, 277.80581039])
cBS = np.array([25.90638071, 23.32572197, 48.87279623, 55.86160267])

scloud = np.zeros(lidar_range_s.shape)
for ci in range(nstruc):
    scloud+=cBS[ci]*np.exp(-(lidar_range_s-calt[ci])**2/csig[ci]**2)

signal = mult*((smol*(1+saer+scloud))/(lidar_range_s**2)+bg_s)

In [ ]:
# output the cloud structure parameters used for the simulation
print(nstruc)
print(calt)
print(csig)
print(cBS)

In [ ]:
# include a realistic overlap function.  Otherwise the first bin dominates the error.
overlap0 = np.array([0,8.26e-6,2.5e-4,1.22e-3,3.46e-3,7.73e-3,3.17e-2,
                     9.09e-2,1.85e-1,3.04e-1,6.02e-1,9.52e-1,9.72e-1,
                     9.85e-1,9.92e-1,1.0,1.0])
overlap_range = np.array([50,100,200,300,400,500,800,1000,1300,1500,2000,3000,4000,5000,6000,8000,12000])
overlap = np.interp(lidar_range_s,overlap_range,overlap0)

signal = signal*overlap

In [ ]:
# generate Poisson observations of the signal
pdemo_s = np.random.poisson(lam=signal)

In [ ]:
# Poisson thin the raw photon counts of the profile
pfit_s,pver_s = poisson_thin(pdemo_s)

In [ ]:
# plot the thinned data.
# the two verify and fit data should have 
# statistically independent noise but common 
# underlying signals
fig,ax = plt.subplots(1,1,figsize=(5,5))
ax.plot(pdemo_s.flatten(),lidar_range_s.flatten()*1e-3,'k.',label='observation')
ax.plot(pfit_s.flatten(),lidar_range_s.flatten()*1e-3,'.',alpha=0.5,label='fit data')
ax.plot(pver_s.flatten(),lidar_range_s.flatten()*1e-3,'.',alpha=0.5,label='verify data')
ax.plot(signal.flatten(),lidar_range_s.flatten()*1e-3,'-',alpha=0.5,label='signal')
ax.set_xscale('log')
ax.set_ylabel('Range [km]')
ax.set_xlabel('Photon Counts')
ax.grid(b=True)
ax.set_ylim([0,12])
ax.legend(loc=1)

In [ ]:
"""
Filter the raw profile
"""

# define range of filters to try
filt_size_s = np.linspace(0.3,3,30)

# initialize the output arrays
inv_ll_s = np.zeros(filt_size_s.size)
inv_ll_rev = np.zeros(filt_size_s.size)
mae_s = np.zeros(filt_size_s.size)
mpe_s = np.zeros(filt_size_s.size)
inv_ll_fit_s = np.zeros(filt_size_s.size)
filt_profs_s = np.zeros((filt_size_s.size,pfit_s.size))

for ifilt,filter_width in enumerate(filt_size_s):
    # get the convolution kernel
    kern = build_Gaussian_kernel(0,filter_width)
    
    # normalize by the amount of points in a region
    # to avoid edge effects
    norm = np.ones(pfit_s.shape)
    norm = scipy.signal.convolve2d(norm,kern,mode='same')
    # apply the filter and normalize the result
    # with the amount of data points contributing
    pfilt = (scipy.signal.convolve2d(pfit_s,kern,mode='same')/norm)
    
    #pfilt[pfilt==0] = 0.001 # avoid zero values that blow up the log function
    
    # save the results
    filt_profs_s[ifilt,:] = pfilt
    inv_ll_s[ifilt] = np.nansum(pfilt-pver_s*np.log(pfilt))  # actual negative log likelihood
    inv_ll_fit_s[ifilt] = np.nansum(pfilt-pfit_s*np.log(pfilt)) # for comparison, also store nLL using fit data as a basis
    inv_ll_rev[ifilt] = np.nansum(0.5*signal-pfilt*np.log(signal)) # for comparison, also store nLL using fit data as a basis

# get the index to the solution
isol_s = np.argmin(inv_ll_s)

In [ ]:
# store the negative log-likelihood of the true signal for comparison later
inv_ll_signal = np.nansum(0.5*signal-pver_s*np.log(0.5*signal))

We plot the results just as before.  The green square shows the optimal filter width as determined by the Poisson thinning with verification technique described previously.  

We also plot the negative log-likelihood computed using fit data (red) instead of the verification data.  Because the noise is correlated, the function is monotonically increasing with increased filter width.  It penalizes stochastic noise suppression.  This is why we need a separate verification profile.

In [ ]:
# plot the resulting optimized profile over the raw data
# also include an over-filtered example
# in a separate plot show the negative log-likelihood as a function of filter width

fig,ax = plt.subplots(2,1,figsize=(5,10))
ax[0].plot(pver_s.flatten(),lidar_range_s.flatten()*1e-3,'.',markersize=2,label='raw data')
ax[0].plot(filt_profs_s[isol_s,:],lidar_range_s.flatten()*1e-3,'k--',label='filtered')
ax[0].plot(0.5*signal.flatten(),lidar_range_s.flatten()*1e-3,':',label='actual')
ax[0].set_xscale('log')
ax[0].set_ylabel('Range [km]')
ax[0].set_xlabel('Photon Counts')
ax[0].grid(b=True)
ax[0].set_ylim([0,12])
ax[0].legend()

ax[1].plot(filt_size_s*dz,inv_ll_s*1e-3,'b.',label='verification nLL')
ax[1].plot(filt_size_s[isol_s]*dz,inv_ll_s[isol_s]*1e-3,'gs',label='minimum')
ax[1].plot(filt_size_s*dz,(inv_ll_fit_s-np.mean(inv_ll_fit_s)+np.mean(inv_ll_s))*1e-3,'rd',alpha=0.5,label='fit nLL')
ax[1].set_xscale('log')
ax[1].grid(b=True)
ax[1].set_xlabel('Filter Width [m]')
ax[1].set_ylabel('Negative Log-Likelihood x $10^{-3}$');

In [ ]:
fig, ax1 = plt.subplots()

ax1.plot(filt_size_s*dz,inv_ll_s*1e-3,'b.')
ax1.plot(filt_size_s*dz,inv_ll_signal*1e-3*np.ones(filt_size_s.shape),'k--')
ax1.grid(b=True)
ax1.set_xlabel('Filter Width [m]')
ax1.set_ylabel('Negative Log-Likelihood x $10^{-3}$',color='blue');

fig, ax1 = plt.subplots()

ax1.plot(filt_size_s*dz,inv_ll_s*1e-3,'b.')
ax1.grid(b=True)
ax1.set_xlabel('Filter Width [m]')
ax1.set_ylabel('Negative Log-Likelihood x $10^{-3}$',color='blue');


In [ ]:
fig,ax1 = plt.subplots(1,1,figsize=(3.1,2.1))
ax1.plot(filt_size_s*dz,inv_ll_s-np.mean(inv_ll_s),'b.')
# ax1.plot(filt_size_s*dz,inv_ll_fit_s-np.mean(inv_ll_fit_s),'rs',markersize=3,alpha=0.6)

ax1.plot(filt_size_s*dz,inv_ll_signal*np.ones(filt_size_s.shape)-np.mean(inv_ll_s),'k--')
ax1.grid(b=True)
ax1.set_xlabel('Filter Width [m]',fontsize=8)
ax1.set_ylabel('$\Delta$ Negative Log-Likelihood',color='black',fontsize=8);

# plt.savefig('../../plots/InvLL_MAE_Filter_SynetheticC0.png',dpi=300,bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(3.2,3.2))
ax.plot(pdemo_s.flatten(),lidar_range_s.flatten()*1e-3,'s',markersize=1,label='observation')
ax.plot(pver_s.flatten(),lidar_range_s.flatten()*1e-3,'d',markersize=1,label='verify data')
ax.plot(pfit_s.flatten(),lidar_range_s.flatten()*1e-3,'.',markersize=1,label='fit data')
ax.plot(filt_profs_s[isol_s,:],lidar_range_s.flatten()*1e-3,'k-',alpha=0.5,label='filtered')
ax.plot(0.5*signal.flatten(),lidar_range_s.flatten()*1e-3,'--',alpha=0.8,label='actual')


ax.set_xscale('log')
ax.set_ylabel('Range [km]')
ax.set_xlabel('Photon Counts')
ax.grid(b=True)
ax.set_ylim([0,12])
ax.legend(loc=0,prop={'size': 6})
# plt.savefig('../../plots/Synthetic_Profile_ComparisonC.png',dpi=300,bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(3.2,3.2))
ax[0].plot(pdemo_s.flatten(),lidar_range_s.flatten()*1e-3,'s',markersize=1,label='obs')
ax[0].plot(pver_s.flatten(),lidar_range_s.flatten()*1e-3,'d',markersize=1,label='verify')
ax[0].plot(pfit_s.flatten(),lidar_range_s.flatten()*1e-3,'.',markersize=1,label='fit')
ax[0].plot(filt_profs_s[isol_s,:],lidar_range_s.flatten()*1e-3,'k-',alpha=0.5,label='opt filter')
ax[0].plot(filt_profs_s[isol_s+5,:],lidar_range_s.flatten()*1e-3,'-',alpha=0.5,label='over filter')
ax[0].plot(0.5*signal.flatten(),lidar_range_s.flatten()*1e-3,'--',label='actual')


ax[1].plot(np.abs(0.5*pdemo_s-0.5*signal).flatten(),lidar_range_s.flatten()*1e-3,'s',markersize=1)
ax[1].plot(np.abs(pver_s-0.5*signal).flatten(),lidar_range_s.flatten()*1e-3,'d',markersize=1)
ax[1].plot(np.abs(pfit_s-0.5*signal).flatten(),lidar_range_s.flatten()*1e-3,'.',markersize=1)
ax[1].plot(np.abs(filt_profs_s[isol_s,:]-0.5*signal.flatten()),lidar_range_s.flatten()*1e-3,'k',alpha=0.5)
ax[1].plot(np.abs(filt_profs_s[isol_s+5,:]-0.5*signal.flatten()),lidar_range_s.flatten()*1e-3,'-',alpha=0.5)
ax[1].set_xlabel('Absolute Error',fontsize=7)

ax[1].grid(b=True)
ax[1].set_ylim([0,12])
labels = [item.get_text() for item in ax[1].get_yticklabels()]
empty_string_labels = ['']*len(labels)
ax[1].set_yticklabels(empty_string_labels)
ax[1].tick_params(axis='x', labelsize=7)

ax[0].set_xscale('log')
ax[0].set_ylabel('Range [km]',fontsize=7)
ax[0].set_xlabel('Photon Counts',fontsize=7)
ax[0].grid(b=True)
ax[0].set_ylim([0,12])
ax[0].legend(loc=0,prop={'size': 5})
ax[0].tick_params(axis='y', labelsize=7)
ax[0].tick_params(axis='x', labelsize=7)
# plt.savefig('../../plots/Synthetic_Profile_Comparison_2panelC.png',dpi=300,bbox_inches='tight')

In [ ]:
i_over = isol_s+5 # index to use as overfitting example

fig,ax = plt.subplots(1,2,figsize=(3.2,3.2))
ax[0].plot(pver_s.flatten(),lidar_range_s.flatten()*1e-3,'d',markersize=1,label='verify')
ax[0].plot(filt_profs_s[isol_s,:],lidar_range_s.flatten()*1e-3,'k-',alpha=0.7,label='opt filter')
ax[0].plot(filt_profs_s[i_over,:],lidar_range_s.flatten()*1e-3,'--',alpha=0.7,label='over filter')
ax[0].plot(0.5*signal.flatten(),lidar_range_s.flatten()*1e-3,':',label='actual')

ax[1].plot(np.abs(pver_s-0.5*signal).flatten(),lidar_range_s.flatten()*1e-3,'d',markersize=1)
ax[1].plot(np.abs(filt_profs_s[isol_s,:]-0.5*signal.flatten()),lidar_range_s.flatten()*1e-3,'k',alpha=0.7)
ax[1].plot(np.abs(filt_profs_s[i_over,:]-0.5*signal.flatten()),lidar_range_s.flatten()*1e-3,'--',alpha=0.7)
ax[1].set_xlabel('Absolute Error',fontsize=7)


ax[1].grid(b=True)
ax[1].set_ylim([0,12])
labels = [item.get_text() for item in ax[1].get_yticklabels()]
empty_string_labels = ['']*len(labels)
ax[1].set_yticklabels(empty_string_labels)
ax[1].tick_params(axis='x', labelsize=7)


ax[0].set_xscale('log')
ax[0].set_ylabel('Range [km]',fontsize=7)
ax[0].set_xlabel('Photon Counts',fontsize=7)
ax[0].grid(b=True)
ax[0].set_ylim([0,12])
ax[0].legend(loc=0,prop={'size': 5})
ax[0].tick_params(axis='y', labelsize=7)
ax[0].tick_params(axis='x', labelsize=7)
# plt.savefig('../../plots/Synthetic_Profile_Comparison_3panelC.png',dpi=300,bbox_inches='tight')

# 2D Optimization for Backscatter Ratio
Principles described above can be similarly applied to multiple channels in two dimensions.  The example below applies filter optimization to estimating backscatter ratio from an HSRL in low signal-to-noise (SNR) conditions.  There is a significant amount of solar background noise in the molecular and cobmined channels of the lidar, it's operating at low pulse energy and the potassium filter used in the molecular channel is less efficient than its rubidium counterpart.  This is a difficult scene to process.

We start by loading both the combined and molecular data.

In [ ]:
"""
Load the photon counts from the netcdf file
"""


profs = {}
with xr.open_dataset(data_path+ncfile) as ds:
    for lvar in lidar_profile_data:
        profs[lvar] = ds[lvar].values
    lidar_range = ds['range'].values.copy()
    time = ds['time'].values.copy()
    

In [ ]:

max_alt = 10e3

imax_alt = np.argmin(np.abs(lidar_range-max_alt))

hsrl_profs = {}
for var in lidar_profile_data:
    # assign the demonstration case to its own independent variable
    ptemp = profs[var]      # remove bottom three bins from laser "bang"
    hsrl_profs[var] = {}
    hsrl_profs[var]['raw'] = ptemp.copy()
    hsrl_profs[var]['bg'] = 0.5*np.mean(hsrl_profs[var]['raw'][:,-100:],axis=1)[:,np.newaxis]
    
    hsrl_profs[var]['raw']=hsrl_profs[var]['raw'][:,3:imax_alt]
    
    # Poisson thin the raw photon counts of the profile
    hsrl_profs[var]['fit'],hsrl_profs[var]['ver'] = poisson_thin(hsrl_profs[var]['raw'])
    
hsrl_lidar_range = lidar_range[3:imax_alt]  # create a new range array for the bang-removed profile
dz = np.mean(np.diff(hsrl_lidar_range))  # store the range resolution  
dt = np.mean(np.diff(time))/np.timedelta64(1, 's')  # store the time resolution  

Ideally we would evaluate all combinations of filters in range and time, but in the interest of speed, faster and more practical to separate the optimization.  So first we optimize the smoothing in time.  Then optimize in range.  Note that the optimization is faster if we evaluate fewer filters.

The optimization is applied independently to both molecular and combined channels.

In general it is a good idea to remove known structure from the signals before applying smoothing to avoid having it limit the amout of smoothing.  For smoothing in time, we remove the background before applying the kernel, then add it back before evaluating the negative log-likelihood.

In [ ]:
# define range of filters to try
time_filt_size = np.logspace(-1,3,100)

# initialize the output arrays
hsrl_inv_ll = {}
hsrl_filt_profs = {}
for ch in hsrl_profs:
    hsrl_inv_ll[ch] = {'time':np.zeros((time_filt_size.size,hsrl_profs[ch]['fit'].shape[1]))}
    hsrl_filt_profs[ch] = np.zeros(hsrl_profs[ch]['fit'].shape+(time_filt_size.size,))

for ifilt,filter_width in enumerate(time_filt_size):
    # get the convolution kernel
    kern = build_Gaussian_kernel(filter_width,0)
    
    # normalize by the amount of points in a region
    # to avoid edge effects
    norm = np.ones(hsrl_profs['Combined_Counts']['fit'].shape)
    norm = scipy.signal.convolve2d(norm,kern,mode='same')
    
    for ch in hsrl_profs:
        # apply the filter and normalize the result
        # with the amount of data points contributing
        # remove the background to avoid having it limit the smoothing kernel
        pfilt = scipy.signal.convolve2d(hsrl_profs[ch]['fit']-hsrl_profs[ch]['bg'],kern,mode='same')/norm
        pfilt+=hsrl_profs[ch]['bg']
        pfilt[pfilt==0] = 0.001 # avoid zero values that blow up the log function

        # save the results
        hsrl_filt_profs[ch][:,:,ifilt] = pfilt.copy()
        hsrl_inv_ll[ch]['time'][ifilt,:] = np.nansum(pfilt-hsrl_profs[ch]['ver']*np.log(pfilt),axis=0)

isol = {}
for ch in hsrl_profs:
    # get the index to the solution
    isol[ch] = {}
    isol[ch]['time'] = np.argmin(hsrl_inv_ll[ch]['time'],axis=0)
    hsrl_profs[ch]['filtered'] = hsrl_profs[ch]['fit'].copy()
    hsrl_profs[ch]['filtered'][:,np.arange(hsrl_filt_profs[ch].shape[1])] = \
                hsrl_filt_profs[ch][:,np.arange(hsrl_filt_profs[ch].shape[1]),isol[ch]['time']]
    

In [ ]:
plt.figure(figsize=(5,5))
for ch in hsrl_profs:
    plt.plot(time_filt_size[isol[ch]['time']]*dt,hsrl_lidar_range,'.',alpha=0.8,label=ch.replace('_Counts',' '))
plt.xlabel('Kernel Width [s]')
plt.ylabel('Altitude [m]')
plt.xscale('log')
plt.grid(b=True)
plt.legend(loc=1)
# plt.savefig('../../plots/Optimized_2D_Filter_Width_Time.png',dpi=300)

The optimal smoothing kernels in time for each range bin are shown above.  Regions with clouds generally require smaller kernels to preserve the signal structure, where relatively clear altitudes use large kernels to suppress shot noise. 

We now optimize the range smoothing on the optimized output from time smoothing.  Similar to what we did when removing background, it can help to remove known structure in range before applying the kernel, then reapplying before evaluating the negative log-likelihood.

In [ ]:
# define range of filters to try
range_filt_size = np.logspace(-1,1.5,40)

# initialize the output arrays
hsrl_filt_profs = {}
for ch in hsrl_profs:
    hsrl_filt_profs[ch] = np.zeros(hsrl_profs[ch]['fit'].shape+(range_filt_size.size,))
    hsrl_inv_ll[ch]['range'] = np.zeros((range_filt_size.size,hsrl_profs[ch]['fit'].shape[0]))

for ifilt,filter_width in enumerate(range_filt_size):
    # get the convolution kernel
    kern = build_Gaussian_kernel(0,filter_width)
    
    # normalize by the amount of points in a region
    # to avoid edge effects
    norm = np.ones(hsrl_profs['Combined_Counts']['fit'].shape)
    norm = scipy.signal.convolve2d(norm,kern,mode='same')
    
    for ch in hsrl_profs:
        # apply the filter and normalize the result
        # with the amount of data points contributing
        # remove the background to avoid having it limit the smoothing kernel
        pfilt = scipy.signal.convolve2d(hsrl_profs[ch]['filtered'],kern,mode='same')/norm
        pfilt[pfilt==0] = 0.001 # avoid zero values that blow up the log function

        # save the results
        hsrl_filt_profs[ch][:,:,ifilt] = pfilt.copy()
        hsrl_inv_ll[ch]['range'][ifilt,:] = np.nansum(pfilt-hsrl_profs[ch]['ver']*np.log(pfilt),axis=1)

for ch in hsrl_profs:
    # get the index to the solution
    isol[ch]['range'] = np.argmin(hsrl_inv_ll[ch]['range'],axis=0)
    hsrl_profs[ch]['filtered'] = hsrl_profs[ch]['fit'].copy()
    hsrl_profs[ch]['filtered'][np.arange(hsrl_filt_profs[ch].shape[0]),:] = \
                hsrl_filt_profs[ch][np.arange(hsrl_filt_profs[ch].shape[0]),:,isol[ch]['range']]
    hsrl_profs[ch]['overfilt'] = hsrl_filt_profs[ch][:,:,-1]

In [ ]:
plt.figure(figsize=(5,5))
for ch in hsrl_profs:
    plt.plot(time,range_filt_size[isol[ch]['range']]*dz,'.',alpha=0.6,label=ch.replace('_Counts',' '))
plt.ylabel('Kernel Width [m]')
plt.xlabel('Time [UTC]')
plt.yscale('log')
plt.grid(b=True)
plt.legend()
# plt.savefig('../../plots/Optimized_2D_Filter_Width_Range.png',dpi=300)

In [ ]:
plt.figure()
for ch in hsrl_profs:
    plt.figure()
    plt.imshow((hsrl_profs[ch]['fit']-hsrl_profs[ch]['bg'])[:,::-1].T,norm=LogNorm())
    plt.clim([1,1e3])
    plt.figure()
    plt.imshow((hsrl_profs[ch]['filtered']-hsrl_profs[ch]['bg'])[:,::-1].T,norm=LogNorm())
    plt.clim([1,1e3])

The plots above show the background subtracted photon counts of unsmoothed and optimally smoothed profiles for both the combined (top two) and molecular channels (bottom two).

In [ ]:
"""
Calculate the backscatter ratio for the difference cases
"""

mol_gain = 2.0
backscatter_ratio_ver = (hsrl_profs['Combined_Counts']['ver']-hsrl_profs['Combined_Counts']['bg'])/ \
                        (hsrl_profs['Molecular_Counts']['ver']-hsrl_profs['Molecular_Counts']['bg'])/mol_gain
backscatter_ratio = (hsrl_profs['Combined_Counts']['fit']-hsrl_profs['Combined_Counts']['bg'])/ \
                        (hsrl_profs['Molecular_Counts']['fit']-hsrl_profs['Molecular_Counts']['bg'])/mol_gain
backscatter_ratio_filtered = (hsrl_profs['Combined_Counts']['filtered']-hsrl_profs['Combined_Counts']['bg'])/ \
                        (hsrl_profs['Molecular_Counts']['filtered']-hsrl_profs['Molecular_Counts']['bg'])/mol_gain
    
backscatter_ratio_overfilt = (hsrl_profs['Combined_Counts']['overfilt']-hsrl_profs['Combined_Counts']['bg'])/ \
                        (hsrl_profs['Molecular_Counts']['overfilt']-hsrl_profs['Molecular_Counts']['bg'])/mol_gain

In [ ]:
plt.figure(figsize=(6,5))
plt.imshow(backscatter_ratio[:,::-1].T,norm=LogNorm())
plt.clim(1,1e2)
plt.figure(figsize=(6,5))
plt.imshow(backscatter_ratio_filtered[:,::-1].T,norm=LogNorm())
plt.clim(1,1e2)
plt.figure(figsize=(6,5))
plt.imshow(backscatter_ratio_overfilt[:,::-1].T,norm=LogNorm())
plt.clim(1,1e2)

The plots above compare the cases where backscatter ratio is estimated with no smoothing (top), optimized smoothing in time and range (middle) and optimized temporal smoothing but over smoothing in range (bottom).  The top and bottom figures show the extremes of error contributions from stochastic noise (top) and smearing (bottom).

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(5,5))
ax.plot(backscatter_ratio[itime,:],hsrl_lidar_range,'.-',alpha=0.5,label='unfiltered')
ax.plot(backscatter_ratio_filtered[itime,:],hsrl_lidar_range,'.-',label='optimized')
ax.plot(backscatter_ratio_overfilt[itime,:],hsrl_lidar_range,'.-',alpha=0.5,label='over filtered')
ax.set_ylim([0,10e3])
ax.set_xlim([1,100])
ax.set_xscale('log')
ax.grid(b=True)
ax.legend()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(3.2,3.2))
ax.plot(backscatter_ratio[itime,:],hsrl_lidar_range*1e-3,'.-',alpha=0.5,markersize=3,label='unfiltered')
ax.plot(backscatter_ratio_filtered[itime,:],hsrl_lidar_range*1e-3,'.-',markersize=3,label='optimized')
ax.plot(backscatter_ratio_overfilt[itime,:],hsrl_lidar_range*1e-3,'.-',markersize=3,alpha=0.5,label='over filtered')
ax.set_ylim([0,10])
ax.set_xlim([1,100])
ax.set_xscale('log')
ax.set_ylabel('Altitude [km]')
ax.set_xlabel('Backscatter Ratio')
ax.grid(b=True)
ax.legend(prop={'size': 8})
# plt.savefig('../../plots/Optimized_2D_BSR_Profile.png',dpi=300,bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(3.2,3.5))
for ch in hsrl_profs:
    ax[0].plot(time,range_filt_size[isol[ch]['range']]*dz,'.',markersize=2,alpha=0.6,label=ch.replace('_Counts',' '))
    ax[1].plot(hsrl_lidar_range,time_filt_size[isol[ch]['time']]*dt,'.',markersize=2,alpha=0.8,label=ch.replace('_Counts',' '))
ax[0].set_ylabel('Kernel Width [m]')
ax[0].set_xlabel('Time [UTC]')
ax[0].tick_params(axis='x', labelsize=7)
ax[0].grid(b=True)
#ax[0].legend(prop={'size': 8})
ax[1].set_ylabel('Kernel Width [s]')
ax[1].set_xlabel('Altitude [m]')
ax[1].tick_params(axis='x', labelsize=7)
ax[1].grid(b=True)
ax[1].legend(loc=7,prop={'size': 8})
# plt.savefig('../../plots/Optimized_2D_Filter_Widths.png',dpi=300)

In [ ]:
plt.figure()
ax1 = plt.subplot2grid((3, 3), (0, 0), colspan=2,rowspan=2)
ax1.pcolor(time,hsrl_lidar_range*1e-3,backscatter_ratio_filtered.T,norm=LogNorm(),vmin=1,vmax=1e2)
ax1.set_ylabel('Altitude [km]')
ax1.set_title('Backscatter Ratio')
ax1.get_xaxis().set_visible(False)
ax1.tick_params(axis='y', labelsize=7)

ax2 = plt.subplot2grid((3, 3), (2, 0), colspan=2)
for ch in hsrl_profs:
    ax2.plot(time,range_filt_size[isol[ch]['range']]*dz,'.',markersize=2,alpha=0.5,label=ch.replace('_Counts',' '))
ax2.set_xlabel('Time [UTC]')
ax2.set_ylabel('Range Resolution [m]')
ax2.grid(b=True,which='both',axis='y')
ax2.grid(b=True,which='major',axis='x')
ax2.tick_params(axis='x', labelsize=7)
ax2.tick_params(axis='y', labelsize=7)
ax2.format_xdata = mdates.DateFormatter('%H:%M')


ax3 = plt.subplot2grid((3, 3), (0, 2), rowspan=2)
for ch in hsrl_profs:
    ax3.plot(time_filt_size[isol[ch]['time']]*dt,hsrl_lidar_range,'.',markersize=2,alpha=0.5,label=ch.replace('_Counts',' '))
ax3.set_xlabel('Time Resolution [s]')
labels = [item.get_text() for item in ax3.get_yticklabels()]
empty_string_labels = ['']*len(labels)
ax3.set_yticklabels(empty_string_labels)

ax3.grid(b=True,which='both',axis='x')
ax3.grid(b=True,which='major',axis='y')


# plt.savefig('../../plots/Optimized_2D_Filter_w_BSR.png',dpi=300)

In [ ]:
ch = 'Combined_Counts'
plt.figure()
ax1 = plt.subplot2grid((6, 16), (0, 0), colspan=8,rowspan=2)
cimg = ax1.pcolor(time,hsrl_lidar_range*1e-3,(hsrl_profs[ch]['fit']-hsrl_profs[ch]['bg']).T,norm=LogNorm(),vmin=1,vmax=1e3)
ax1.set_ylabel('Altitude [km]',fontsize=7)
ax1.get_xaxis().set_visible(False)
ax1.tick_params(axis='y', labelsize=7)


ax2 = plt.subplot2grid((6, 16), (2, 0), colspan=8,rowspan=2)
ax2.pcolor(time,hsrl_lidar_range*1e-3,(hsrl_profs[ch]['filtered']-hsrl_profs[ch]['bg']).T,norm=LogNorm(),vmin=1,vmax=1e3)
ax2.set_ylabel('Altitude [km]',fontsize=7)
ax2.get_xaxis().set_visible(False)
ax2.tick_params(axis='y', labelsize=7)


ax4 = plt.subplot2grid((6, 16), (4, 0), colspan=8,rowspan=1)
ax4.plot(time,range_filt_size[isol[ch]['range']]*dz,'.',markersize=2,alpha=0.5,label=ch.replace('_Counts',' '))
ax4.set_xlabel('Time [UTC]',fontsize=7)
ax4.set_ylabel('Filter\nWidth [m]',fontsize=7)
ax4.grid(b=True,which='both',axis='y')
ax4.grid(b=True,which='major',axis='x')
ax4.tick_params(axis='x', labelsize=7)
ax4.tick_params(axis='y', labelsize=7)
ax4.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax4.set_xlim(ax1.get_xlim())
ax4.set_yscale('log')


ax3 = plt.subplot2grid((6, 16), (2, 8), rowspan=2, colspan=3)
ax3.plot(time_filt_size[isol[ch]['time']]*dt,hsrl_lidar_range*1e-3,'.',markersize=2,alpha=0.5,label=ch.replace('_Counts',' '))
ax3.set_xlabel('Filter\nWidth [s]',fontsize=7)
labels = [item.get_text() for item in ax3.get_yticklabels()]
empty_string_labels = ['']*len(labels)
ax3.set_yticklabels(empty_string_labels)
ax3.tick_params(axis='x', labelsize=7)
#ax3.get_yaxis().set_visible(False)
ax3.set_ylim(ax2.get_ylim())
ax3.set_xscale('log')
ax3.xaxis.set_major_locator(ticker.LogLocator(base=10.0,numticks=15))


ax3.grid(b=True,which='both',axis='x')
ax3.grid(b=True,which='major',axis='y')

cax = plt.subplot2grid((6, 8), (0, 4), rowspan=2, colspan=1)
cb = plt.colorbar(cimg, cax = cax)
cb.set_label('Photon Counts',fontsize=7)
cb.ax.tick_params(labelsize=7) 

# plt.savefig('../../plots/Optimized_2D_Filter_'+ch+'2.png',dpi=300,bbox_inches='tight')

